In [ ]:
## Reverse Geocode Steps

# Step One

In [1]:
# load libraries

import requests
import pandas as pd

In [2]:
## Step One ## Using SVI variables to reverse code census tract id
# read abcd svi data
abcd_svi = pd.read_csv('./data/led_l_svi.csv')

# read CDC SVI data
CDC_svi = pd.read_csv('./data/SVI_2018_US.csv')
# https://www.atsdr.cdc.gov/placeandhealth/svi/documentation/SVI_documentation_2018.html

# select the variables to match
# Columns to match for obtaining unique census tract
abcd_columns_id = [
    'reshist_addr1_svi_pov_20142018',
    'reshist_addr1_svi_emp_20142018',
    'reshist_addr1_svi_cap_20142018',
    'reshist_addr1_svi_hs_20142018',
    'reshist_addr1_svi_65_20142018',
    'reshist_addr1_svi_17_20142018',
    'reshist_addr1_svi_dis_20142018',
    'reshist_addr1_svi_sin_20142018',
    'reshist_addr1_svi_min_20142018',
    'reshist_addr1_svi_eng_20142018',
    'reshist_addr1_svi_hous20142018',
    'reshist_addr1_svi_mob_20142018',
    'reshist_addr1_svi_crwd20142018',
    'reshist_addr1_svi_veh_20142018',
    'reshist_addr1_svi_grp_20142018',
    'src_subject_id'  
]

abcd_columns = [
    'reshist_addr1_svi_pov_20142018',
    'reshist_addr1_svi_emp_20142018',
    'reshist_addr1_svi_cap_20142018',
    'reshist_addr1_svi_hs_20142018',
    'reshist_addr1_svi_65_20142018',
    'reshist_addr1_svi_17_20142018',
    'reshist_addr1_svi_dis_20142018',
    'reshist_addr1_svi_sin_20142018',
    'reshist_addr1_svi_min_20142018',
    'reshist_addr1_svi_eng_20142018',
    'reshist_addr1_svi_hous20142018',
    'reshist_addr1_svi_mob_20142018',
    'reshist_addr1_svi_crwd20142018',
    'reshist_addr1_svi_veh_20142018',
    'reshist_addr1_svi_grp_20142018'
]


cdc_columns = [
    'EPL_POV',
    'EPL_UNEMP',
    'EPL_PCI',
    'EPL_NOHSDP',
    'EPL_AGE65',
    'EPL_AGE17',
    'EPL_DISABL',
    'EPL_SNGPNT',
    'EPL_MINRTY',
    'EPL_LIMENG',
    'EPL_MUNIT',
    'EPL_MOBILE',
    'EPL_CROWD',
    'EPL_NOVEH',
    'EPL_GROUPQ',
    'STATE','COUNTY','FIPS','LOCATION','AREA_SQMI'
]

# ABCD selected with id
abcd_svi_selected_id = abcd_svi[abcd_columns_id]

# Select relevant columns from ABCD SVI dataset
abcd_svi_selected = abcd_svi[abcd_columns]

# Select relevant columns from CDC SVI dataset
cdc_svi_selected = CDC_svi[cdc_columns]

# Rename columns in ABCD SVI to match CDC SVI columns
abcd_svi_selected.columns = cdc_columns[:-5]



In [3]:
# Check for uniqueness
matching_columns = [
    'EPL_POV',
    'EPL_UNEMP',
    'EPL_PCI',
    'EPL_NOHSDP',
    'EPL_AGE65',
    'EPL_AGE17',
    'EPL_DISABL',
    'EPL_SNGPNT',
    'EPL_MINRTY',
    'EPL_LIMENG',
    'EPL_MUNIT',
    'EPL_MOBILE',
    'EPL_CROWD',
    'EPL_NOVEH',
    'EPL_GROUPQ'
]
unique_tracts = CDC_svi.drop_duplicates(subset=matching_columns)
print("Number of unique census tracts based on matching columns:", len(unique_tracts))
print("Total number of records in CDC SVI dataset:", len(CDC_svi))

if len(unique_tracts) == len(CDC_svi):
    print("Each combination of selected columns corresponds to a unique FIPS ID.") # 72405
else:
    print("There are non-unique combinations of selected columns.") # 72837 
    
    # Identify non-unique combinations
duplicates = CDC_svi[CDC_svi.duplicated(subset=matching_columns, keep=False)]

# Print the non-unique combinations
print("Non-unique combinations of selected columns:")
print(duplicates[matching_columns + ['FIPS']])

# Save the duplicates to a CSV file
duplicates.to_csv('./data/cdc_svi_duplicates.csv', index=False)


Number of unique census tracts based on matching columns: 72405
Total number of records in CDC SVI dataset: 72837
There are non-unique combinations of selected columns.
Non-unique combinations of selected columns:
       EPL_POV  EPL_UNEMP  EPL_PCI  EPL_NOHSDP  EPL_AGE65  EPL_AGE17  \
0       -999.0     -999.0   -999.0      -999.0     -999.0  -999.0000   
1       -999.0     -999.0   -999.0      -999.0     -999.0  -999.0000   
2       -999.0     -999.0   -999.0      -999.0     -999.0  -999.0000   
4       -999.0     -999.0   -999.0      -999.0     -999.0  -999.0000   
5       -999.0     -999.0   -999.0      -999.0     -999.0  -999.0000   
...        ...        ...      ...         ...        ...        ...   
34592      0.0        0.0   -999.0         0.0        0.0     0.0000   
37481      0.0        0.0   -999.0         0.0        0.0     0.0000   
37651      0.0        0.0   -999.0         0.0        0.0     0.9993   
38691      0.0        0.0   -999.0         0.0        0.0     0.00

In [4]:
# Merge the datasets based on the selected columns to get FIPS (GEOID)
merged_df = pd.merge(abcd_svi_selected_id, cdc_svi_selected, left_on=abcd_columns, right_on=cdc_columns[:-5], how='left')

# Reorder columns to place 'src_subject_id' first
final_columns = ['src_subject_id'] + [col for col in merged_df.columns if col != 'src_subject_id']
merged_df = merged_df[final_columns]

## check if any of the FIPS were the same as FIPS from the duplicates 
# Check for overlapping FIPS codes between unique and duplicate entries
overlap_fips = duplicates['FIPS'].isin(merged_df['FIPS']).any()

if overlap_fips:
    print("There are overlapping FIPS codes between the duplicate and merged datasets.")
else:
    print("No overlapping FIPS codes found.")

# Optionally, to view the specific overlapping FIPS codes
if overlap_fips:
    overlapping_fips = duplicates[duplicates['FIPS'].isin(merged_df['FIPS'])]['FIPS']
    print("Overlapping FIPS codes:", overlapping_fips.unique())





No overlapping FIPS codes found.


In [5]:
# No replicates, then save the merged file 
merged_df.to_csv('./data/merged_abcd_svi_with_fips.csv', index=False)

In [ ]:
# Step Two 

In [6]:
## Using walkability index and gross residential density to locate census block group within the census tract 
## 
import geopandas as gpd
import pandas as pd
import requests
import numpy as np
import fiona

In [7]:
# get the gross population density 
gdb = './SLD/SMARTLOCATIONDB/SmartLocationDb.gdb'
fc = gpd.read_file(gdb)
# Specify the correct layer name
layer_name = 'TIGER2010_bg_SLD'
# Read the specified layer
fc = gpd.read_file(gdb, layer=layer_name)

# Display the first few rows of the DataFrame
print(fc.head())

        GEOID10  TRFIPS CFIPS SFIPS   CSA CSA_Name     CBSA       CBSA_Name  \
0  010059505002  950500   005    01  None     None  21640.0  Eufaula, AL-GA   
1  010059505001  950500   005    01  None     None  21640.0  Eufaula, AL-GA   
2  010059502001  950200   005    01  None     None  21640.0  Eufaula, AL-GA   
3  010059502002  950200   005    01  None     None  21640.0  Eufaula, AL-GA   
4  010059504002  950400   005    01  None     None  21640.0  Eufaula, AL-GA   

   CBSA_EMP  CBSA_POP  ...  D5cri  D5ce  D5cei     D5dr    D5dri     D5de  \
0    9921.0   29970.0  ...   0.66  4.24   0.68 -99999.0 -99999.0 -99999.0   
1    9921.0   29970.0  ...   0.67  4.70   0.75 -99999.0 -99999.0 -99999.0   
2    9921.0   29970.0  ...   0.30  2.60   0.42 -99999.0 -99999.0 -99999.0   
3    9921.0   29970.0  ...   0.27  2.58   0.41 -99999.0 -99999.0 -99999.0   
4    9921.0   29970.0  ...   0.32  3.35   0.54 -99999.0 -99999.0 -99999.0   

     D5dei  Shape_Length  Shape_Area  \
0 -99999.0      0.8469

In [ ]:
# Optionally 

## check the gross residential density for Geoid 
# geoid of 601441800 look at all dataset that start with this geoid 
# Filter the DataFrame for GEOID that starts with '601441800'
#filtered_df = fc[fc['GEOID10'].str.startswith('06001441800')]

# Filter the DataFrame for GEOID10 that starts with '06'
filtered_df = fc[fc['GEOID10'].str.startswith('06001441800')]

# Display the first few rows to inspect similar patterns
print(filtered_df[['GEOID10']].head(10))

# Display the filtered DataFrame
# Set Pandas to display all columns
pd.set_option('display.max_columns', None)

# Display the first few rows of the filtered DataFrame to inspect all columns
print(filtered_df.head(10))

## Matched !!! 

In [9]:
# Check how many of the census block groups have the same d1a within the census tract (first 10 digits identical geoid)
fc['GEOID10_10'] = fc['GEOID10'].str[:10]

# Step 2: Group by the first 10 digits of GEOID10 and check for duplicate D1A values
duplicate_d1a_within_groups = fc[fc.duplicated(subset=['GEOID10_10', 'D1A'], keep=False)]
print("\nRows with Duplicate D1A values within the same 10-digit GEOID groups:")
print(duplicate_d1a_within_groups)

len(duplicate_d1a_within_groups) # 508 duplicates

duplicates_with_zero_d1a = duplicate_d1a_within_groups[duplicate_d1a_within_groups['D1A'] == 0]

count_zero_d1a = duplicates_with_zero_d1a.shape[0]
print("\nNumber of rows with D1A value of 0 within the duplicates:")
print(count_zero_d1a) # 508 were all 0


Rows with Duplicate D1A values within the same 10-digit GEOID groups:
             GEOID10  TRFIPS CFIPS SFIPS   CSA CSA_Name     CBSA   CBSA_Name  \
6977    040190005001  000500   019    04  None     None  46060.0  Tucson, AZ   
7073    040190005005  000500   019    04  None     None  46060.0  Tucson, AZ   
7276    040279800031  980003   027    04  None     None  49740.0    Yuma, AZ   
7365    040279800041  980004   027    04  None     None  49740.0    Yuma, AZ   
7382    040279800051  980005   027    04  None     None  49740.0    Yuma, AZ   
...              ...     ...   ...   ...   ...      ...      ...         ...   
220648  691209501021  950102   120    69  None     None      0.0        None   
220649  691209501011  950101   120    69  None     None      0.0        None   
220650  691209502003  950200   120    69  None     None      0.0        None   
220651  691209502002  950200   120    69  None     None      0.0        None   
220652  691209502001  950200   120    69  None   

In [10]:
## import walkability
# Make sure to use excel sheet and make sure the geoids were in 12 digits format

walk = pd.read_excel('./data/WalkIndex_USBlkGrps.xlsx')

walk['BlkGrpID'] = walk['BlkGrpID'].astype(str)
# Filter the DataFrame for GEOID10 that starts with '06'
filtered_walk = walk[walk['BlkGrpID'].str.startswith('6001441800')]

# Display the first few rows to inspect similar patterns
print(filtered_walk[['BlkGrpID']].head(10))

print(filtered_walk.head(10)) # matched

len(walk) #220334

print(walk.head())

          BlkGrpID
11133  60014418001
11134  60014418002
11135  60014418003
11136  60014418004
          BlkGrpID  StCoFIPS StAbbr  NatWalkInd  Pop2010  HU2010  HH2010  \
11133  60014418001      6001     CA      16.167     2595     911     867   
11134  60014418002      6001     CA      16.167     1235     421     410   
11135  60014418003      6001     CA      14.667     2069     799     756   
11136  60014418004      6001     CA      12.667      887     282     277   

       D2A_EPHHM  D2B_E8MIXA      D3B         D4A  D2A_Ranked  D2B_Ranked  \
11133   0.496664    0.847142  134.269  820.482232          11          20   
11134   0.485309    0.571528  138.259  238.591937          11          12   
11135   0.464759    0.587045  108.353  649.157072          10          12   
11136   0.336812    0.000000  124.141  711.533729           7           1   

       D3B_Ranked  D4A_Ranked  
11133          18          15  
11134          18          19  
11135          17          16  
11136     

In [11]:
fc['GEOID10'] = fc['GEOID10'].astype(str)
walk['BlkGrpID'] = walk['BlkGrpID'].astype(str)


# Remove decimal points from 'BlkGrpID' and convert to string
walk['BlkGrpID'] = walk['BlkGrpID'].apply(lambda x: str(int(float(x))))
walk['BlkGrpID'] = walk['BlkGrpID'].str.zfill(12)
print(walk.head())


#fc['GEOID10'] = fc['GEOID10'].str.zfill(12)
print(fc.head())

# For 'walk', assuming 'BlkGrpID' is the ID column
max_walk_id = walk['BlkGrpID'].max()

# For 'fc', assuming 'GEOID10' is the ID column
max_fc_id = fc['GEOID10'].max()

# Print the maximum IDs
print(f"Maximum ID for walk: {max_walk_id}")
print(f"Maximum ID for fc: {max_fc_id}")

       BlkGrpID  StCoFIPS StAbbr  NatWalkInd  Pop2010  HU2010  HH2010  \
0  010010201001      1001     AL       7.000      698     283     262   
1  010010201002      1001     AL       7.833     1214     469     431   
2  010010202001      1001     AL       7.500     1003     375     341   
3  010010202002      1001     AL      10.167     1167     447     402   
4  010010203001      1001     AL       3.167     2549     963     915   

   D2A_EPHHM  D2B_E8MIXA      D3B      D4A  D2A_Ranked  D2B_Ranked  \
0   0.729616    0.544996  11.4319 -99999.0          17          11   
1   0.573856    0.913864  13.0227 -99999.0          13          20   
2   0.628905    0.499410  37.3242 -99999.0          15          10   
3   0.741931    0.729355  59.2237 -99999.0          18          17   
4   0.125196    0.000000  24.0513 -99999.0           2           1   

   D3B_Ranked  D4A_Ranked  
0           6           1  
1           6           1  
2           9           1  
3          12           1  


In [12]:
## Combine Walkability and D1a based on BlkGrpID (Walk) and ID (fc)
len(fc) #220653
len(walk) #220334

# make sure to retain most of the IDs, so left join onto fc


220334

In [13]:
# Select only BlkGrpID and NatWalkInd from walk
walk_reduced = walk[['BlkGrpID', 'NatWalkInd']]

# Select only GEOID10 and D1A from fc
fc_reduced = fc[['GEOID10', 'D1A']]

# delete states that start with 02, any state >= 55
# Filter out states starting with '02' and any state with codes >= 55
#fc_reduced = fc_reduced[~(fc_reduced['GEOID10'].str.startswith('02') | (fc_reduced['GEOID10'].str[:2].astype(int) >= 55))]

# How many of them matched 
# Perform the left join
combined_data = fc_reduced.merge(walk_reduced, how='left', left_on='GEOID10', right_on='BlkGrpID')

# export the walk_reduced in csv
#walk_reduced.to_csv('walk_reduced.csv', index=False)  # Set index=False to not include row indices in the file

# Add a column to check if the IDs match
combined_data['ID_Match'] = combined_data['BlkGrpID'] == combined_data['GEOID10']

# Count matching and non-matching records
matching_count = combined_data['ID_Match'].sum()
non_matching_count = (~combined_data['ID_Match']).sum()

print(f"Number of matching IDs: {matching_count}")
print(f"Number of non-matching IDs: {non_matching_count}")

# Display non-matching records
non_matching_records = combined_data[~combined_data['ID_Match']]

print("Non-matching Records:")
print(non_matching_records[['GEOID10', 'BlkGrpID', 'D1A', 'NatWalkInd']])

# print out the first two digits of the nonmatching records
# Extract the first two digits of GEOID10 for non-matching records
non_matching_records['StateCode'] = non_matching_records['GEOID10'].str[:2]

# Extract unique state codes from non-matching records
unique_state_codes = non_matching_records['StateCode'].unique()

# Print the unique state codes
print("Unique state codes from non-matching records:")
print(unique_state_codes)

Number of matching IDs: 220263
Number of non-matching IDs: 390
Non-matching Records:
             GEOID10 BlkGrpID       D1A  NatWalkInd
3447    022700001001      NaN  0.000235         NaN
3448    022700001004      NaN  0.000225         NaN
3449    022700001003      NaN  0.000498         NaN
3450    022700001002      NaN  0.000081         NaN
6404    040194704002      NaN  0.013269         NaN
...              ...      ...       ...         ...
220648  691209501021      NaN  0.000000         NaN
220649  691209501011      NaN  0.000000         NaN
220650  691209502003      NaN  0.000000         NaN
220651  691209502002      NaN  0.000000         NaN
220652  691209502001      NaN  0.000000         NaN

[390 rows x 4 columns]
Unique state codes from non-matching records:
['02' '04' '06' '36' '46' '51' '60' '66' '69']


C:\Users\qyuan25\AppData\Local\Temp\ipykernel_44748\2630858197.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matching_records['StateCode'] = non_matching_records['GEOID10'].str[:2]


In [14]:
# Perform the left join
combined_data = fc_reduced.merge(walk_reduced, how='left', left_on='GEOID10', right_on='BlkGrpID')
# Display the result
print("Combined Data:")
print(combined_data.head())

# Check the lengths to ensure all records from fc are retained
print(f"\nLength of fc: {len(fc)}")
print(f"Length of combined data: {len(combined_data)}")

Combined Data:
        GEOID10       D1A      BlkGrpID  NatWalkInd
0  010059505002  0.011064  010059505002       3.000
1  010059505001  0.084972  010059505001       4.500
2  010059502001  0.008673  010059502001       5.667
3  010059502002  0.006218  010059502002       3.833
4  010059504002  0.021627  010059504002       5.667

Length of fc: 220653
Length of combined data: 220653


In [15]:
## Check how many of them has been connected
# Check successful and unsuccessful matches
successful_matches = combined_data['NatWalkInd'].notna().sum()
unsuccessful_matches = combined_data['NatWalkInd'].isna().sum()

# Display the results
print(f"Number of successful matches: {successful_matches}")
print(f"Number of unsuccessful matches (missing walkability data): {unsuccessful_matches}")

Number of successful matches: 220263
Number of unsuccessful matches (missing walkability data): 390


In [16]:
## import ABCD walkability and gross residential density index 

abcd_walk = pd.read_csv('./data/led_l_walk.csv')

abcd_d1a = pd.read_csv('./data/led_l_densbld.csv')

abcd_fips = pd.read_csv('./data/merged_abcd_svi_with_fips.csv')

print("Column names and count for abcd_walk:")
print(abcd_walk.columns)
print("Total columns:", len(abcd_walk))

print("\nColumn names and count for abcd_d1a:")
print(abcd_d1a.columns)
print("Total columns:", len(abcd_d1a))

print("\nColumn names and count for abcd_fips:")
print(abcd_fips.columns)
print("Total columns:", len(abcd_fips))

Column names and count for abcd_walk:
Index(['src_subject_id', 'eventname', 'reshist_addr1_walkindex',
       'reshist_addr2_walkindex', 'reshist_addr3_walkindex'],
      dtype='object')
Total columns: 11216

Column names and count for abcd_d1a:
Index(['src_subject_id', 'eventname', 'reshist_addr1_d1a', 'reshist_addr2_d1a',
       'reshist_addr3_d1a'],
      dtype='object')
Total columns: 11216

Column names and count for abcd_fips:
Index(['src_subject_id', 'STATE', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI',
       'reshist_addr1_svi_pov_20142018', 'reshist_addr1_svi_emp_20142018',
       'reshist_addr1_svi_cap_20142018', 'reshist_addr1_svi_hs_20142018',
       'reshist_addr1_svi_65_20142018', 'reshist_addr1_svi_17_20142018',
       'reshist_addr1_svi_dis_20142018', 'reshist_addr1_svi_sin_20142018',
       'reshist_addr1_svi_min_20142018', 'reshist_addr1_svi_eng_20142018',
       'reshist_addr1_svi_hous20142018', 'reshist_addr1_svi_mob_20142018',
       'reshist_addr1_svi_crwd20142018'

In [17]:
## left_join these three files based on src_subject_ID
# Performing the left joins
merged_df = abcd_fips.merge(abcd_walk, on='src_subject_id', how='left')
merged_df = merged_df.merge(abcd_d1a, on='src_subject_id', how='left')

# If you want to check the resulting DataFrame
print("Column names in the merged DataFrame:")
print(merged_df.columns)
print("Total columns:", len(merged_df.columns))

Column names in the merged DataFrame:
Index(['src_subject_id', 'STATE', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI',
       'reshist_addr1_svi_pov_20142018', 'reshist_addr1_svi_emp_20142018',
       'reshist_addr1_svi_cap_20142018', 'reshist_addr1_svi_hs_20142018',
       'reshist_addr1_svi_65_20142018', 'reshist_addr1_svi_17_20142018',
       'reshist_addr1_svi_dis_20142018', 'reshist_addr1_svi_sin_20142018',
       'reshist_addr1_svi_min_20142018', 'reshist_addr1_svi_eng_20142018',
       'reshist_addr1_svi_hous20142018', 'reshist_addr1_svi_mob_20142018',
       'reshist_addr1_svi_crwd20142018', 'reshist_addr1_svi_veh_20142018',
       'reshist_addr1_svi_grp_20142018', 'EPL_POV', 'EPL_UNEMP', 'EPL_PCI',
       'EPL_NOHSDP', 'EPL_AGE65', 'EPL_AGE17', 'EPL_DISABL', 'EPL_SNGPNT',
       'EPL_MINRTY', 'EPL_LIMENG', 'EPL_MUNIT', 'EPL_MOBILE', 'EPL_CROWD',
       'EPL_NOVEH', 'EPL_GROUPQ', 'eventname_x', 'reshist_addr1_walkindex',
       'reshist_addr2_walkindex', 'reshist_addr3_walkindex', 

In [18]:
print("Total columns:", len(merged_df))
merged_df.head()

Total columns: 11232


,src_subject_id,STATE,COUNTY,FIPS,LOCATION,AREA_SQMI,reshist_addr1_svi_pov_20142018,reshist_addr1_svi_emp_20142018,reshist_addr1_svi_cap_20142018,reshist_addr1_svi_hs_20142018,...,EPL_NOVEH,EPL_GROUPQ,eventname_x,reshist_addr1_walkindex,reshist_addr2_walkindex,reshist_addr3_walkindex,eventname_y,reshist_addr1_d1a,reshist_addr2_d1a,reshist_addr3_d1a
0,NDAR_INVRUVAZC2F,ARIZONA,Maricopa,4.013817e+09,"Census Tract 8169, Maricopa County, Arizona",9.429805,0.2554,0.3544,0.3086,0.0597,...,0.1080,0.0000,baseline_year_1_arm_1,7.000,7.000,NaN,baseline_year_1_arm_1,0.070879,0.070879,NaN
1,NDAR_INV65V1Z314,ARKANSAS,Pope,5.115951e+09,"Census Tract 9512, Pope County, Arkansas",84.775572,0.6130,0.2996,0.5735,0.5116,...,0.0476,0.0000,baseline_year_1_arm_1,5.667,5.667,NaN,baseline_year_1_arm_1,0.059590,0.059590,NaN
2,NDAR_INV1T61M03G,CALIFORNIA,Alameda,6.001428e+09,"Census Tract 4276, Alameda County, California",0.225045,0.7083,0.1238,0.5585,0.8326,...,0.7307,0.5476,baseline_year_1_arm_1,15.667,NaN,NaN,baseline_year_1_arm_1,17.218853,NaN,NaN
3,NDAR_INV8AWV40W3,CALIFORNIA,Alameda,6.001441e+09,"Census Tract 4413.02, Alameda County, California",0.770145,0.4783,0.4346,0.1128,0.2822,...,0.0476,0.6536,baseline_year_1_arm_1,14.833,14.833,NaN,baseline_year_1_arm_1,6.103608,6.103608,NaN
4,NDAR_INV75VFMGNT,CALIFORNIA,Alameda,6.001442e+09,"Census Tract 4415.01, Alameda County, California",1.013511,0.1209,0.3138,0.1571,0.1545,...,0.0334,0.4400,baseline_year_1_arm_1,8.500,NaN,NaN,baseline_year_1_arm_1,5.063131,NaN,NaN


In [21]:
## before merging, check the duplicates within combined_data

# Check for duplicates based on 'CensusTractFIPS', 'NatWalkInd', and 'D1A'
duplicates = combined_data.duplicated(subset=['GEOID10', 'NatWalkInd', 'D1A'], keep=False)

# Display all duplicate rows to understand the issue
duplicate_rows = combined_data[duplicates]
print("Duplicate rows based on 'CensusTractFIPS', 'NatWalkInd', and 'D1A':")
print(duplicate_rows) # 312 duplicates

# write combined_data in csv
#combined_data.to_csv('data/combined_walk_d1a_data.csv', index=False)

# write duplicate_rows in csv
duplicate_rows.to_csv('./data/duplicate_walk_d1a_data.csv',index=False)

# write merged_df in csv
merged_df.to_csv('data/merged_walk_d1a_abcd.csv', index=False)

Duplicate rows based on 'CensusTractFIPS', 'NatWalkInd', and 'D1A':
Empty DataFrame
Columns: [GEOID10, D1A, BlkGrpID, NatWalkInd]
Index: []


In [ ]:
# import the source data and abcd data
merged_abcd = pd.read_csv('data/merged_walk_d1a_abcd.csv')
combined_data = pd.read_csv('data/combined_walk_d1a_data.csv')


In [34]:
# Confirm column presence and create if necessary
if 'FIPS' in merged_abcd.columns:
    merged_abcd['CensusTractFIPS'] = merged_abcd['FIPS'].astype(str).str[:11]

# Check that the column exists now
print(merged_abcd.columns)

Index(['src_subject_id', 'STATE', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI',
       'reshist_addr1_svi_pov_20142018', 'reshist_addr1_svi_emp_20142018',
       'reshist_addr1_svi_cap_20142018', 'reshist_addr1_svi_hs_20142018',
       'reshist_addr1_svi_65_20142018', 'reshist_addr1_svi_17_20142018',
       'reshist_addr1_svi_dis_20142018', 'reshist_addr1_svi_sin_20142018',
       'reshist_addr1_svi_min_20142018', 'reshist_addr1_svi_eng_20142018',
       'reshist_addr1_svi_hous20142018', 'reshist_addr1_svi_mob_20142018',
       'reshist_addr1_svi_crwd20142018', 'reshist_addr1_svi_veh_20142018',
       'reshist_addr1_svi_grp_20142018', 'EPL_POV', 'EPL_UNEMP', 'EPL_PCI',
       'EPL_NOHSDP', 'EPL_AGE65', 'EPL_AGE17', 'EPL_DISABL', 'EPL_SNGPNT',
       'EPL_MINRTY', 'EPL_LIMENG', 'EPL_MUNIT', 'EPL_MOBILE', 'EPL_CROWD',
       'EPL_NOVEH', 'EPL_GROUPQ', 'eventname_x', 'reshist_addr1_walkindex',
       'reshist_addr2_walkindex', 'reshist_addr3_walkindex', 'eventname_y',
       'reshist_addr1_d

In [36]:
merged_abcd['CensusTractFIPS'] = merged_abcd['CensusTractFIPS'].astype(str)
combined_data['CensusTractFIPS'] = combined_data['CensusTractFIPS'].astype(str)


In [38]:

# Rename columns in combined_data for consistency in merging
combined_data.rename(columns={'NatWalkInd': 'reshist_addr1_walkindex', 'D1A': 'reshist_addr1_d1a'}, inplace=True)

# Drop rows with NaN in the key columns
merged_abcd.dropna(subset=['reshist_addr1_walkindex', 'reshist_addr1_d1a'], inplace=True)
combined_data.dropna(subset=['reshist_addr1_walkindex', 'reshist_addr1_d1a'], inplace=True)

# Round the values to set a tolerance of 0.001
tolerance = 0.001
merged_abcd['reshist_addr1_walkindex'] = merged_abcd['reshist_addr1_walkindex'].round(4)
merged_abcd['reshist_addr1_d1a'] = merged_abcd['reshist_addr1_d1a'].round(4)
combined_data['reshist_addr1_walkindex'] = combined_data['reshist_addr1_walkindex'].round(4)
combined_data['reshist_addr1_d1a'] = combined_data['reshist_addr1_d1a'].round(4)

# Create a function to perform the custom merge
def merge_with_tolerance(df1, df2, tol, on):
    df1['_key'] = 1
    df2['_key'] = 1
    merged = pd.merge(df1, df2, on=['_key', 'CensusTractFIPS'])
    merged = merged[(abs(merged['reshist_addr1_walkindex_x'] - merged['reshist_addr1_walkindex_y']) <= tol) &
                    (abs(merged['reshist_addr1_d1a_x'] - merged['reshist_addr1_d1a_y']) <= tol)]
    merged = merged.drop(columns=['_key', 'reshist_addr1_walkindex_y', 'reshist_addr1_d1a_y'])
    merged = merged.rename(columns={'reshist_addr1_walkindex_x': 'reshist_addr1_walkindex', 'reshist_addr1_d1a_x': 'reshist_addr1_d1a'})
    return merged

# Perform the merge with tolerance
result = merge_with_tolerance(merged_abcd, combined_data, tolerance, on=['CensusTractFIPS', 'reshist_addr1_walkindex', 'reshist_addr1_d1a'])

# Count the number of matches
num_matches = result.shape[0]

print(f"Number of matching rows: {num_matches}")

# Check the number of NAs in key fields after merging
num_nas = result['GEOID10'].isna().sum()
print("Number of NAs in key fields after merging:", num_nas)

# Filter rows with NAs in GEOID10
nas_in_geo_id = result[result['GEOID10'].isna()]



Number of matching rows: 7978
Number of NAs in key fields after merging: 0


In [31]:
len(result)

0

In [39]:
# Print columns to confirm 'CensusTractFIPS' exists in both dataframes
print("Columns in merged_abcd:")
print(merged_abcd.columns)

print("Columns in combined_data:")
print(combined_data.columns)

print(result.columns)

Columns in merged_abcd:
Index(['src_subject_id', 'STATE', 'COUNTY', 'FIPS', 'LOCATION', 'AREA_SQMI',
       'reshist_addr1_svi_pov_20142018', 'reshist_addr1_svi_emp_20142018',
       'reshist_addr1_svi_cap_20142018', 'reshist_addr1_svi_hs_20142018',
       'reshist_addr1_svi_65_20142018', 'reshist_addr1_svi_17_20142018',
       'reshist_addr1_svi_dis_20142018', 'reshist_addr1_svi_sin_20142018',
       'reshist_addr1_svi_min_20142018', 'reshist_addr1_svi_eng_20142018',
       'reshist_addr1_svi_hous20142018', 'reshist_addr1_svi_mob_20142018',
       'reshist_addr1_svi_crwd20142018', 'reshist_addr1_svi_veh_20142018',
       'reshist_addr1_svi_grp_20142018', 'EPL_POV', 'EPL_UNEMP', 'EPL_PCI',
       'EPL_NOHSDP', 'EPL_AGE65', 'EPL_AGE17', 'EPL_DISABL', 'EPL_SNGPNT',
       'EPL_MINRTY', 'EPL_LIMENG', 'EPL_MUNIT', 'EPL_MOBILE', 'EPL_CROWD',
       'EPL_NOVEH', 'EPL_GROUPQ', 'eventname_x', 'reshist_addr1_walkindex',
       'reshist_addr2_walkindex', 'reshist_addr3_walkindex', 'eventname_y',

In [ ]:
# check any of the CensusTractFIPS were the same as the ones in the duplicates



In [40]:
# Identify rows in merged_abcd that were not matched in the result
merged_abcd['_merge_key'] = merged_abcd['CensusTractFIPS'].astype(str) + '-' + merged_abcd['reshist_addr1_walkindex'].astype(str) + '-' + merged_abcd['reshist_addr1_d1a'].astype(str)
result['_merge_key'] = result['CensusTractFIPS'].astype(str) + '-' + result['reshist_addr1_walkindex'].astype(str) + '-' + result['reshist_addr1_d1a'].astype(str)

# Find rows in merged_abcd not present in the result
unmatched = merged_abcd[~merged_abcd['_merge_key'].isin(result['_merge_key'])]

# Drop the _merge_key column before saving
unmatched = unmatched.drop(columns=['_merge_key'])

# Save the unmatched rows to a CSV file
unmatched.to_csv('unmatched_abcd.csv', index=False)
print(f"Rows not matched have been saved to unmatched_abcd.csv. Total unmatched rows: {len(unmatched)}")

Rows not matched have been saved to unmatched_abcd.csv. Total unmatched rows: 3240


In [8]:
## save the combined_data_duplicates to csv
# Find duplicate rows based on the specified columns
duplicates = combined_data[combined_data.duplicated(subset=['src_subject_id', 'reshist_addr1_walkindex', 'reshist_addr1_d1a'], keep=False)]

duplicates.to_csv('./data/second_duplicates.csv',index=False)

len(duplicates)#34

# Check if any FIPS from result occur in duplicates
result_fips_in_duplicates = result['CensusTractFIPS'].isin(duplicates['CensusTractFIPS'])

# Filter to show only the rows from result that have FIPS occurring in duplicates
fips_in_duplicates = result[result_fips_in_duplicates]

# Display or save the filtered data
print(fips_in_duplicates)

## NONE of the Result were in Duplicates

NameError: name 'combined_data' is not defined

In [ ]:
# for those unmatched
## try another merge only using the first two digits of the FIPS and 'reshist_addr1_walkindex', 'reshist_addr1_d1a'

len(unmatched) ## 481

unmatched['CensusTractFIPS'] = unmatched['CensusTractFIPS'].astype(str)
combined_data['CensusTractFIPS'] = combined_data['CensusTractFIPS'].astype(str)

# Round the values to set a tolerance of 0.001
unmatched['reshist_addr1_walkindex'] = unmatched['reshist_addr1_walkindex'].round(4)
unmatched['reshist_addr1_d1a'] = unmatched['reshist_addr1_d1a'].round(4)
combined_data['reshist_addr1_walkindex'] = combined_data['reshist_addr1_walkindex'].round(4)
combined_data['reshist_addr1_d1a'] = combined_data['reshist_addr1_d1a'].round(4)

# Extract the first two digits of the FIPS
unmatched['CensusTractFIPS_short'] = unmatched['CensusTractFIPS'].str[:2]
combined_data['CensusTractFIPS_short'] = combined_data['CensusTractFIPS'].str[:2]

# Create a function to perform the custom merge
def merge_with_tolerance(df1, df2, tol):
    df1['_key'] = 1
    df2['_key'] = 1
    merged = pd.merge(df1, df2, on=['_key', 'CensusTractFIPS_short'])
    merged = merged[(abs(merged['reshist_addr1_walkindex_x'] - merged['reshist_addr1_walkindex_y']) <= tol) &
                    (abs(merged['reshist_addr1_d1a_x'] - merged['reshist_addr1_d1a_y']) <= tol)]
    merged = merged.drop(columns=['_key', 'reshist_addr1_walkindex_y', 'reshist_addr1_d1a_y'])
    merged = merged.rename(columns={'reshist_addr1_walkindex_x': 'reshist_addr1_walkindex', 'reshist_addr1_d1a_x': 'reshist_addr1_d1a'})
    return merged

# perform the merge with tolerance using shortened FIPS
result_short_fips = merge_with_tolerance(unmatched, combined_data, tolerance)

# Check the number of matches
num_matches_short_fips = result_short_fips.shape[0]
print(f"Number of matching rows with short FIPS: {num_matches_short_fips}")

# Identify rows in unmatched that were still not matched
unmatched['_merge_key'] = unmatched['CensusTractFIPS_short'].astype(str) + '-' + unmatched['reshist_addr1_walkindex'].astype(str) + '-' + unmatched['reshist_addr1_d1a'].astype(str)
result_short_fips['_merge_key'] = result_short_fips['CensusTractFIPS_short'].astype(str) + '-' + result_short_fips['reshist_addr1_walkindex'].astype(str) + '-' + result_short_fips['reshist_addr1_d1a'].astype(str)

# Find rows in unmatched not present in result_short_fips
still_unmatched = unmatched[~unmatched['_merge_key'].isin(result_short_fips['_merge_key'])]

# Drop the _merge_key column before saving
still_unmatched = still_unmatched.drop(columns=['_merge_key'])

# Save the still unmatched rows to a CSV file
still_unmatched.to_csv('still_unmatched_abcd.csv', index=False)
print(f"Rows still not matched have been saved to still_unmatched_abcd.csv. Total still unmatched rows: {len(still_unmatched)}")

In [ ]:
## Find duplicated rows for third match (third step)
third_duplicates = combined_data[combined_data.duplicated(subset=['CensusTractFIPS_short', 'reshist_addr1_walkindex', 'reshist_addr1_d1a'], keep=False)]

third_duplicates.to_csv('./data/third_duplicates_short.csv',index=False)
len(third_duplicates) # 3233

result_short_fips_in_duplicates = result_short_fips['GEOID10'].isin(second_duplicates['GEOID10'])

    # Filter to show only the rows from result_short_fips that have GEOID10 occurring in duplicates
fips_in_duplicates_short = result_short_fips[result_short_fips_in_duplicates]

    # Display or save the filtered data
print(fips_in_duplicates_short) # none of the fips in second_match were in the duplicates. 

result_short_fips.to_csv('second_match_state_abcd.csv',index=False)


In [ ]:
# STEP FOUR 

# for those still unmatched (NAs for svi-derived fips)
# try merging with combined_data only using reshist_addr1_walkindex and reshist_addr1_d1a and site state

site_id = pd.read_excel('./data/subject_site_id.xlsx')

# merge site_id and result_walkindex_d1a using src_subject_id
# Merge the DataFrames on the 'src_subject_id' column
result_site = pd.merge(site_id, still_unmatched, on='src_subject_id', how='inner')

# Round the values to set a tolerance of 0.001
result_site['reshist_addr1_walkindex'] = result_site['reshist_addr1_walkindex'].round(4)
result_site['reshist_addr1_d1a'] = result_site['reshist_addr1_d1a'].round(4)
result_site['CensusTractFIPS_short'] = result_site['site_state']

# Convert the types of CensusTractFIPS_short to ensure they match
result_site['CensusTractFIPS_short'] = result_site['CensusTractFIPS_short'].astype(str)
combined_data['CensusTractFIPS_short'] = combined_data['CensusTractFIPS_short'].astype(str)

tolerance=0.001

# Create a function to perform the custom merge based only on walkindex and d1a
def merge_with_result_site(df1, df2, tol):
    df1['_key'] = 1
    df2['_key'] = 1
    merged = pd.merge(df1, df2, on=['_key', 'CensusTractFIPS_short'])
    merged = merged[(abs(merged['reshist_addr1_walkindex_x'] - merged['reshist_addr1_walkindex_y']) <= tol) &
                    (abs(merged['reshist_addr1_d1a_x'] - merged['reshist_addr1_d1a_y']) <= tol)]
    merged = merged.drop(columns=['_key', 'reshist_addr1_walkindex_y', 'reshist_addr1_d1a_y'])
    merged = merged.rename(columns={'reshist_addr1_walkindex_x': 'reshist_addr1_walkindex', 'reshist_addr1_d1a_x': 'reshist_addr1_d1a'})
    return merged

In [ ]:
# Perform the merge with tolerance using only walkindex and d1a
merge_3 = merge_with_result_site(result_site, combined_data, tolerance)

# Check the number of matches
num_matches_merge_3 = merge_3.shape[0]
print(f"Number of matching rows with walkindex and d1a: {num_matches_merge_3}")



In [ ]:
# remove the duplicates and save to merge_3_no_duplicates
# Remove duplicates
merge_3_in_no_duplicates = merge_3.drop_duplicates()
merge_3_in_no_duplicates.to_csv('./data/merge_3_no_duplicates.csv',index=False)



In [ ]:
## Any still unmatched after the third match?, 
match1 = result
match2 = result_short_fips
match3 = merge_3_in_no_duplicates

# all matches
all_matches = pd.concat([match1['src_subject_id'], match2['src_subject_id'], match3['src_subject_id']]).drop_duplicates()

# Identify unmatched src_subject_id values
unmatched_ids = merged_df[~merged_df['src_subject_id'].isin(all_matches)]

# Display the number of matched and unmatched ids
num_matched = all_matches.shape[0]
num_unmatched = unmatched_ids.shape[0]

print(f"Number of matched src_subject_id values: {num_matched}")
print(f"Number of unmatched src_subject_id values: {num_unmatched}")

# Optionally, save the unmatched src_subject_id values to a CSV file
unmatched_ids.to_csv('unmatched_src_subject_id.csv', index=False) # 103 remains
# successfully matched 11113 participants

In [ ]:
## merge three matches together 
# Combine matched datasets by merging on 'src_subject_id'
combined_horizontal = pd.concat([match1, match2, match3], axis=0)

# Print the column names
print(combined_horizontal.columns.tolist())

In [ ]:
# Retain only the specified columns
columns_to_retain = [
  'src_subject_id', 'STATE', 'COUNTY', 'CensusTractFIPS', 'LOCATION', 'AREA_SQMI',
'BlkGrpID', '_merge_key', 'CensusTractFIPS_x', 'CensusTractFIPS_short', 
  'CensusTractFIPS_y', 'site', 'site_state', 'site_state_name'
]

## Check filtered_combined_matches (any duplicates of src_subject_id)
duplicates = filtered_combined_matches[filtered_combined_matches.duplicated(subset='src_subject_id', keep=False)]

# Print the number of duplicate rows and the duplicate rows themselves
num_duplicates = duplicates.shape[0]
print(f"Number of duplicate src_subject_id values: {num_duplicates}")
print(duplicates)

no_duplicates_combined_matches = filtered_combined_matches.drop_duplicates(subset='src_subject_id', keep=False)
len(no_duplicates_combined_matches) # 11098 

# save the final dataset
no_duplicates_combined_matches.to_csv('abcd_blkgrp_matches_nodup.csv', index=False)
